# SETUP

In [27]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
import os
HOME = os.getcwd()
import shutil

/content


In [ ]:
!pip install ultralytics

import ultralytics
from ultralytics import YOLO



In [ ]:
!pip install supervision


import supervision as sv


In [5]:
import numpy as np

In [6]:
import scipy.io as sio
import cv2

In [7]:
import pandas as pd

# GOOGLE DRIVE MOUNT

In [8]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/"

# RESTORE DATA

In [25]:
%cd "/content"

!cp "/content/drive/MyDrive/Colab Notebooks/gt.txt" .

trackers = ["bytetrack", "deepsort", "sort", "ocsort", "botsort", "strongsort"]


for tracker in trackers:
    source_path = f"/content/drive/MyDrive/Colab Notebooks/{tracker}/{tracker}_results.txt"
    !cp "{source_path}" .


/content


In [11]:
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/MVI_1482_VIS.avi"

# TRACK EVALUATION


## Install TrackEval

### Clean Download

In [ ]:
%cd /content/
%rm -fr TrackEval


import sys
import importlib

sys.modules.pop('trackeval', None)
for module in list(sys.modules.keys()):
    if module.startswith('trackeval'):
        sys.modules.pop(module, None)
!git clone https://github.com/JonathonLuiten/TrackEval.git
%cd TrackEval

### Fix the old numpy compatibility

In [ ]:
# To be updated files
files_to_fix = [
    '/content/TrackEval/trackeval/metrics/hota.py',
    '/content/TrackEval/trackeval/metrics/clear.py',
    '/content/TrackEval/trackeval/metrics/identity.py',
    '/content/TrackEval/trackeval/metrics/vace.py',
    '/content/TrackEval/trackeval/datasets/mot_challenge_2d_box.py'
]

# Updates
replacements = {
    'np.int': 'np.int64',
    'np.float': 'np.float64',
    'dtype=float': 'dtype=np.float64',
    'dtype=int': 'dtype=np.int64'
}

for file_path in files_to_fix:
    try:
        with open(file_path, 'r') as file:
            content = file.read()

        for old, new in replacements.items():
            content = content.replace(old, new)

        with open(file_path, 'w') as file:
            file.write(content)

        print(f"Successfully modified: {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

print("\nAll files have been processed.")

### Install

In [ ]:
%cd /content/TrackEval/
!pip install -e .

### Import


In [15]:
import trackeval

In [ ]:

# Copy Ground truth file
!cp "/content/gt.txt" "/content/TrackEval/data/mot_challenge/custom/MVI_1482_VIS/gt/gt.txt"

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
seqinfo_content = f"""[Sequence]
name=MVI_1482_VIS
imDir=img1
frameRate={video_info.fps}
seqLength={video_info.total_frames}
imWidth={video_info.width}
imHeight={video_info.height}
imExt=.jpg"""

with open("/content/TrackEval/data/mot_challenge/custom/MVI_1482_VIS/seqinfo.ini", 'w') as f:
    f.write(seqinfo_content)

# Dosyaların varlığını kontrol edelim
print("\nChecking files:")
!ls -R /content/TrackEval/data/mot_challenge/custom/


# Ensure the seqmaps directory exists
os.makedirs('/content/TrackEval/data/mot_challenge/custom/seqmaps', exist_ok=True)

# Define the content of the sequence map file
seqmap_content = """name
MVI_1482_VIS
"""

# Write the sequence map file
seqmap_path = '/content/TrackEval/data/mot_challenge/custom/seqmaps/custom-train.txt'
with open(seqmap_path, 'w') as f:
    f.write(seqmap_content)

# Verify the file exists
assert os.path.isfile(seqmap_path), "Sequence map file was not created successfully."

## Evaluate

In [26]:
def setup_mot_evaluation(video_path, algorithm_name="bytetrack"):
    %cd /content/

    video_name = os.path.basename(video_path).split('.')[0]
    base_path = "/content/TrackEval/data"

    # check files
    gt_source = f"gt.txt"
    tracker_source = f"{algorithm_name}_results.txt"
    if not os.path.exists(gt_source):
        raise FileNotFoundError(f"Cloudn't find ground truth file: {gt_source}")
    if not os.path.exists(tracker_source):
        raise FileNotFoundError(f"Could not find tracker results file: {tracker_source}")


    # Main paths
    paths = {
        'seqmaps': f"{base_path}/gt/mot_challenge/seqmaps",
        'gt_base': f"{base_path}/gt/mot_challenge/mot_challenge-train/{video_name}",
        'gt_data': f"{base_path}/gt/mot_challenge/mot_challenge-train/{video_name}/gt",
        'tracker': f"{base_path}/trackers/mot_challenge/mot_challenge-train/{algorithm_name}/data"
    }


    # Make dirs
    for path in paths.values():
        os.makedirs(path, exist_ok=True)

    # Copy Ground truth file
    gt_target = os.path.join(paths['gt_data'], gt_source)
    shutil.copy2(gt_source, gt_target)

    # Copy tracking results
    tracker_target = os.path.join(paths['tracker'], f"{video_name}.txt")
    shutil.copy2(tracker_source, tracker_target)

    # seqinfo.ini
    video_info = sv.VideoInfo.from_video_path(video_path)
    seqinfo_content = f"""[Sequence]
name={video_name}
imDir=img1
frameRate={video_info.fps}
seqLength={video_info.total_frames}
imWidth={video_info.width}
imHeight={video_info.height}
imExt=.jpg"""

    seqinfo_path = os.path.join(paths['gt_base'], 'seqinfo.ini')
    with open(seqinfo_path, 'w') as f:
        f.write(seqinfo_content)

    # sequence map file
    seqmap_content = f"""name
{video_name}"""

    seqmap_path = os.path.join(paths['seqmaps'], 'mot_challenge-train.txt')
    with open(seqmap_path, 'w') as f:
        f.write(seqmap_content)

    print("Files created successfully.")
    print("\nFiles:")
    print(f"GT file: {os.path.exists(gt_target)}")
    print(f"Tracker file: {os.path.exists(tracker_target)}")
    print(f"Seqinfo file: {os.path.exists(seqinfo_path)}")
    print(f"Seqmap file: {os.path.exists(seqmap_path)}")

    if not os.path.exists(gt_target):
        print(f"\nWarning: Ground truth file not found at {gt_target}")
        print(f"Source: {gt_source}")
        print(f"Target: {gt_target}")
        print(f"Source File? : {os.path.exists(gt_source)}")
        print(f"Target path? : {os.path.exists(os.path.dirname(gt_target))}")


In [ ]:
for algorithm in trackers:
    # Start setup
    setup_mot_evaluation(
        video_path=SOURCE_VIDEO_PATH,
        algorithm_name=algorithm
    )

    # Evaluate
    %cd /content/TrackEval/
    !python scripts/run_mot_challenge.py \
        --BENCHMARK mot_challenge \
        --SPLIT_TO_EVAL train \
        --TRACKERS_TO_EVAL {algorithm} \
        --DO_PREPROC False